In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Text Preprocessing Methods

import unidecode
import pandas as pd
import re
import time
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
import timeit

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import sklearn
import pandas as pd
import numpy as np
import csv
import os
import random
import torch
import torch
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
import sklearn.model_selection as model_selection
from transformers import BertForSequenceClassification, BertTokenizerFast
import time
import copy
from tqdm.notebook import tqdm

In [4]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [5]:
set_seed()

In [6]:
train = pd.read_csv('/kaggle/input/amazon-ml-challenge/train.csv')
train

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/amazon-ml-challenge/train.csv'

In [ ]:
test = pd.read_csv('/kaggle/input/amazon-ml-challenge/train.csv')
test

In [ ]:
train.info()

In [ ]:
def count_null_values(df, dataset_name):
    
    num_of_total_null_values = sum(df.isnull().sum().values)
    print(f"{dataset_name} dataset has {num_of_total_null_values} null values")
    return num_of_total_null_values

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train_null = count_null_values(train, "train")

In [ ]:
import matplotlib.pyplot as plt

def desc(s):
    print("NULL : " + str(s.isnull().sum()))
    print("un : " + str(s.nunique()))
    print(s.describe())

In [ ]:
desc(train['PRODUCT_TYPE_ID'])


In [ ]:
desc(train['TITLE'])

In [ ]:
desc(train['DESCRIPTION'])

In [ ]:
desc(train['BULLET_POINTS'])

In [ ]:
desc(train['PRODUCT_LENGTH'])



In [ ]:
Only_title = pd.DataFrame(train[['TITLE', 'PRODUCT_LENGTH']])

In [ ]:
type(Only_title['TITLE'])

In [ ]:
def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text

def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text  

def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text

def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text




In [ ]:
def text_preprocessing(text, accented_chars=True, extra_whitespace=True,
                       newlines_tabs=True, repeatition=True, punctuations=True,  
                       special_chars=True):
    """
    This function will preprocess input text and return
    the clean text.
    """
        
    if newlines_tabs == True: #remove newlines & tabs.
        Data = remove_newlines_tabs(text)
        
    if extra_whitespace == True: #remove extra whitespaces
        Data = remove_whitespace(Data)
        
    if accented_chars == True: #remove accented characters
        Data = accented_characters_removal(Data)
        
    if repeatition == True: #Reduce repeatitions   
        Data = reducing_incorrect_character_repeatation(Data)
    
    if punctuations == True: #remove punctuations
        Data = removing_special_characters(Data)
    
    if punctuations == True: #remove punctuations
        Data = removing_special_characters(Data)
           
    return Data

In [ ]:
List_Content = Only_title['TITLE'].to_list()
Final_Article = []
Complete_Content = []
for article in List_Content:
    Processed_Content = text_preprocessing(str(article)) #Cleaned text of Content attribute after pre-processing
    Final_Article.append(Processed_Content)
Complete_Content.extend(Final_Article)
Only_title['Processed_title'] = Complete_Content

In [ ]:
Cleaned_only_title = Only_title.dropna()
Cleaned_only_title['TITLE'] = Cleaned_only_title['Processed_title']
Cleaned_only_title = Cleaned_only_title.drop(['Processed_title'],axis = 1)

In [ ]:
Cleaned_only_title.to_csv('/kaggle/working/title.csv')

In [ ]:
def Remove_outliers(df):
    percentile25 = df['PRODUCT_LENGTH'].quantile(0.25)
    percentile75 = df['PRODUCT_LENGTH'].quantile(0.75)
    Q1 = np.percentile(df['PRODUCT_LENGTH'], 25,
                       interpolation = 'midpoint')

    Q3 = np.percentile(df['PRODUCT_LENGTH'], 75,
                       interpolation = 'midpoint')
    iqr = Q3 - Q1
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    
    return df[df['PRODUCT_LENGTH'] < upper_limit]

In [ ]:
Cleaned_only_title = Remove_outliers(Cleaned_only_title)


In [ ]:
def create_folds(data, num_splits):
    data["kfold"] = -1
    data = data.sample(frac=1).reset_index(drop=True)
    y=Cleaned_only_title["PRODUCT_LENGTH"]
    kf = model_selection.KFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=y)):
        data.loc[v_, 'kfold'] = f
    return data

In [ ]:
Cleaned_only_title_folds=create_folds(Cleaned_only_title, 5)

In [ ]:
Cleaned_only_title_folds02=Cleaned_only_title_folds.loc[Cleaned_only_title_folds.kfold.isin([0,2])]
Cleaned_only_title_folds02=Cleaned_only_title_folds02.reset_index(drop=True)
Cleaned_only_title_folds02

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')



In [ ]:
seq_len = [len(i.split()) for i in Cleaned_only_title_folds02['TITLE']]
pd.Series(seq_len).hist(bins = 40)

In [ ]:
pd.DataFrame(seq_len).describe()

In [ ]:
max_seq_len = 25

In [ ]:
encoded_corpus = tokenizer(text=Cleaned_only_title_folds02['TITLE'].tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_seq_len,
                            return_attention_mask=True)

In [ ]:
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

In [ ]:
test_size = 0.1
seed = 42
train_inputs, test_inputs, train_target, test_target = train_test_split(input_ids, np.array(Cleaned_only_title_folds02['PRODUCT_LENGTH']), test_size=test_size, 
                             random_state=seed)
train_masks, test_masks, _, _ = train_test_split(attention_mask, 
                                        np.array(Cleaned_only_title_folds02['PRODUCT_LENGTH']), test_size=test_size, 
                                        random_state=seed)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
batch_size = 32
def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor, 
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=True)
    return dataloader
train_dataloader = create_dataloaders(train_inputs, train_masks, 
                                      train_target, batch_size)
test_dataloader = create_dataloaders(test_inputs, test_masks, 
                                     test_target, batch_size)

In [ ]:
 out = BertForSequenceClassification.from_pretrained('bert-base-uncased')(torch.tensor(input_ids), torch.tensor(attention_mask))

In [ ]:
class BERTRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_camembert=False):
        
        super(BERTRegressor, self).__init__()
        D_in, D_out = 768, 1
        
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_mask):
        
        outputs = self.bert(torch.tensor(input_ids), torch.tensor(attention_mask))
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs

In [ ]:
model = BERTRegressor(drop_rate=0.2)

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)

In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,       
                 num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
loss_function = nn.MSELoss()

In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm
def train(model, optimizer, scheduler, loss_function, epochs,       
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader): 
            print(step)  
            batch_inputs, batch_masks, batch_labels = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)           
            loss = loss_function(outputs.squeeze(), 
                             batch_labels.squeeze())
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()
                
    return model
model = train(model, optimizer, scheduler, loss_function, epochs, 
              train_dataloader, device, clip_value=2)

In [ ]:
outputs = BertForSequenceClassification.from_pretrained('bert-base-uncased')(torch.tensor(input_ids), torch.tensor(attention_mask))